### Part 1: Import Libraries & Data

In [142]:
# Import Libraries
import csv
import pandas as pd 
import numpy as np
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Import dataset
data = pd.read_csv('wine.csv')

In [144]:
# This group was causing errors before (NaNs in "type"), so this is a check
# to make sure the issue is fixed. 
data[13998:14002]

,ything,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,type
13998,18882.0,South Africa,"A Bordeaux-style blend of 56% Merlot, 24% Cabe...",Z,90.0,35.0,Stellenbosch,NaN,NaN,Lauren Buzzeo,@laurbuzz,De Toren 2009 Z Red,Bordeaux-style Red Blend,De Toren,Red Blend
13999,18883.0,Spain,This blend of 66% Touriga Nacional and one-thi...,Cuvée TNS,90.0,144.0,Central Spain,Manchuela,NaN,Michael Schachner,@wineschach,Finca Sandoval 2007 Cuvée TNS Red,Touriga Nacional Blend,Finca Sandoval,White Blend
14000,18887.0,US,A stand-out wine among Keuka Lake Vineyard's m...,Estate Grown and Bottled Falling Man Vineyard Dry,90.0,30.0,New York,Finger Lakes,Finger Lakes,Anna Lee C. Iijima,NaN,Keuka Lake Vineyards 2011 Estate Grown and Bot...,Riesling,Keuka Lake Vineyards,White
14001,18891.0,Spain,This multigrape blend opens with spicy aromas ...,#NAME?,90.0,32.0,Central Spain,Dominio de Valdepusa,NaN,Michael Schachner,@wineschach,Marques de Griñon 2007 + Summa Varietals Syra...,Red Blend,Marques de Griñon,Red Blend


### Part 2: Tokenize description boxes. 

In [145]:
# Break up the sentences into lists of individual words
for i in range(len(data)):
    text = data['description'][i]
    data['description'][i] = word_tokenize(text.lower())

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [146]:
from nltk.corpus import stopwords
a = set(stopwords.words("english"))

for i in range(len(data)):
    data['description'][i] = [x for x in data['description'][i] if x not in a]

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [147]:
# Part-of-Speech Tagging
# Definitions: https://www.guru99.com/pos-tagging-chunking-nltk.html
# NN is the main part of speech we want to keep here. Everything else can go. 
# - NN = singular

# Add the type of speech to each word in the list. 
for i in range(len(data)):
    words = []
    for token in data['description'][i]:
        words.append(nltk.pos_tag([token]))
        data['description'][i] = words       

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [148]:
for i in range(len(data)):
    new_list = []
    final_set = []
    for tag in range(len(data['description'][i])):
        if data['description'][i][tag][0][1] == 'NN' or data['description'][i][tag][0][1] == 'JJ':
            final_set.append(data['description'][i][tag][0][0])
    data['description'][i] = final_set

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### Part 3: Match with Flavor Categories

In [149]:
# Create new box, "category"
data['category'] = np.empty((len(data), 0)).tolist()

In [150]:
# Source: https://winefolly.com/deep-dive/wine-tasting-terms-to-use/
# Source 2: https://www.decanter.com/learn/advice/understand-tasting-notes-decoded-344920/#fruit

# ========== Flavors ============ #

# Fruity
fruit_forward_red = ["Berry", 'black-cherry', "Raspberry", "Cherry", "Coconut", "Cassis", 'red-berry', "Blackberry", "Blueberry", "Jam", "Prune", "Candied", "Raisin", "Toffee", "Vanilla", "Bergamot", "Olive", "Bramble", "Cranberry", "Fig" , "Jammy", "Juniper" , "Kirsch", "Loganberry" , "Plum", "Raisin", "Raspberry" , "Strawberry", "Fruit", "Fruity"]
fruit_forward_white = ["Lemon", "Apple", "Peach", "Mango", "Pear", "Berry", "Cantaloupe", "Creme Brulee", "Crème Brûlée", "Caramel", "Vanilla", "Apricot", "Banana", "Candied", "Citrus", "Honey", "Gooseberry", "Kiwi", "Lychee", "Marmalade", "Melon", "Orange", "Papaya", "Passion Fruit", "Pineapple", "Prune", "Sherbet", "Fruit", "Fruity"]

# Savory 
savory_red = ["Savory", "Cranberry", "Soy", "Onion", "Rhubarb"," Black Currant", "Cassis", "Pepper", 'lemon-zest', "Peppercorn", "Olive", "Mulberry", "Bilberry", "Dried Herbs", "Game", "Sage", "Leather", "Tobacco", "Charcoal", "Tar", "Underbrush", "Garrigue", "Gravel", "Torrefaction", "Mineral", "Woodsmoke"]
savory_white = ["Savory", "Lime", "Pith", "Quince", "Almond", "Gooseberry", "Jalapeno", "Grapefruit", "Papaya", "Thyme", "Chervil", "Grass", "Flint", "Chalk", "Chalky", "Petrichor", "Minerally", "Mineral", "biscuit", "brioche", "buttery", "butter", "caramel", "cereal", "cream", "marzipan", "croissant", "pastry"]

# Earthy
earthy_red = ["Earthy", "Rough", "Tannic", "Rusty", "Rustic", 'lead', "Earthy", "Balsamic", 'herbal', 'woody','spicy', 'clove', "Eucalyptus", "Pepper", "Leafy", "Medicinal", "Mint", "Mushroom", "Rhubarb", "Tomato", "beetroot", "tea", "meat", "tobacco", "cardboard", "iodine", "charcoal", "chocolate", "coffee", "leather","tar", "smoke", "wood", "vinyl", "velvet", "velvety", "Pepper", "Spice", "Spices", "Cedar", "Cinnamon", "Clove", "Cola", "Cumin", "Licorice"]
earthy_white = ["Earthy", "Asparagus", "Cabbage", "Fennel", "Grass", "Hay", "Hedgerow", "Lemongrass", 'herbal', 'woody','spicy', 'clove', "Vegetal", "Chalky", "flint", "chalk", "graphite", "mineral", "oyster", "salt", "slate", "steely", "wool", "almond", "beeswax", "petrol", "gasoline", "smoky", "Rough", "Tannic", "Rusty", "Rustic", "Earthy", "smoke", "toffee", "vanilla", "walnut", "wax", "match", "Pepper", "Spice", "Spices", "Cedar", "Cinnamon"]
                      
# Floral 
floral_red = ["Floral", "Blossom", "Rosy", "Rose", 'fragrant', "Lavender", "Peony", "Flower", "Flowery", "Rose", "Turkish Delight", "Violet"]
floral_white = ["Floral", "Blossom", 'Camomile', "Geranium", "Elderflower", 'fragrant', "Honeysuckle", "Jasmine", "Ginger", "Flower", "Flowery", "Rosy", "Rose"]
  
# Bitter 
bitter_red = ["Chewy", "Muscular", "Structured", "Firm", "Rigid", "Closed", "Dried Herbs", "Herby", "Oregano", "Bay Leaf", "Bitter Chocolate", "Baker’s Chocolate", "Bitter Herbs", "Austere", "Angular", "Grippy", "Harsh", "Coarse", "Dense"]
bitter_white = ["Austere", "Citrus Pith", "Quince", "Bitter", "Almond", "Green", "Almond", "Chalk", "Chalky"]

# ========== Body Profile ============ #
# Light-Bodied
light_bodied_red = ["Light-bodied", 'summer', "Light", "fresh:, ""Subtle", "Delicate", "Elegant", "Crisp", "Thin", "Finesse", "Bright", "Floral"]
light_bodied_white = ["Light-bodied", 'summer', "Light","Zesty", "Airy", "Lean", "fresh", "Racy", "Crisp", "Zippy", "Austere", "Long Tingly Finish", "Brilliant", "Lively"]

# Full-Bodied
full_bodied_red = ["Full-bodied", "Rich", "Lush", "Opulent", 'richness', 'syrah', "Rigid", "Intense", "Extracted", "High Alcohol", "High Tannin", "Firm", "Structured", "Muscular", "Concentrated", "Hot",'ripe', 'luscious','heft', 'bold', 'lavish']
full_bodied_white = ["Full-bodied", "Rich", "Lush", "Oily", "Buttery", 'richness', "biscuit", "brioche", "buttery", "butter", "caramel", "shortcake", "cereal", "cream", "marzipan", "croissant", "pastry",'ripe', 'luscious','heft', 'bold', 'lavish']


In [151]:
# Put in lowercase: 
fruit_forward_red = [x.lower() for x in fruit_forward_red]
fruit_forward_white = [x.lower() for x in fruit_forward_white]

savory_red = [x.lower() for x in savory_red]
savory_white = [x.lower() for x in savory_white]

earthy_red = [x.lower() for x in earthy_red]
earthy_white = [x.lower() for x in earthy_white]

floral_red = [x.lower() for x in floral_red]
floral_white = [x.lower() for x in floral_white]

bitter_red = [x.lower() for x in bitter_red]
bitter_white = [x.lower() for x in bitter_white]

light_bodied_red = [x.lower() for x in light_bodied_red]
light_bodied_white = [x.lower() for x in light_bodied_white]

full_bodied_red = [x.lower() for x in full_bodied_red]
full_bodied_white = [x.lower() for x in full_bodied_white]

In [152]:
# Turns out there were some "nulls" in the "type" column. 
# Taking those outs allows for 
data = data[data['type'].notnull()]

In [155]:
for i in range(len(data)):
    
    # ======================== Red Wines ================================================================
    if ' Red ' in data["type"][i].split():
    # =====================================================
    # Fruity Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in fruit_forward_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Fruity")
    # =====================================================
    # Savory Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in savory_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory") 
            
    # =====================================================
    # Earthy Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in earthy_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Earthy") 
    # =====================================================
    # Floral Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in floral_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Floral") 
            
    # =====================================================
    # Bitter Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in bitter_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Bitter") 
            
    # =====================================================
    # Light-Bodied Red
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in light_bodied_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Light-Bodied")
            
    # =====================================================
    # Full-Bodied Red
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in full_bodied_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Full-Bodied")
            
            
    # ======================== White Wines ================================================================
    elif ' White ' in data["type"][i].split():
    # =====================================================
    # Fruity Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in fruit_forward_white:
                  sum += 1
        if sum >= 2:
            data["category"][i].append("Fruity")

    # =====================================================
    # Savory Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in savory_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory")
    # =====================================================   
    # Earthy Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in earthy_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Earthy") 
    # =====================================================
    # Floral Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in floral_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Floral") 
        
    # =====================================================
    # Bitter Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in bitter_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Bitter") 
            
    # =====================================================
    # Light-Bodied White
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in light_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Light-Bodied")   
    # =====================================================
    # Full-Bodied White
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in full_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Full-Bodied") 
            
  # ======================== Rosé and Blends ================================================================
    else: 
        # Fruity
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in fruit_forward_red or data["description"][i][j] in fruit_forward_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Fruity")
            
        # Savory 
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in savory_red or data["description"][i][j] in savory_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory") 
            
       # Earthy 
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in earthy_red or data["description"][i][j] in earthy_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Earthy") 
              
        # Floral
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in floral_red or data["description"][i][j] in floral_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Floral") 
        
        # Bitter
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in bitter_red or data["description"][i][j] in bitter_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Bitter") 
            
        # Light-Bodied
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in light_bodied_red or data["description"][i][j] in light_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Light-Bodied")      
                 
        # Full-Bodied 
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in full_bodied_red or data["description"][i][j] in full_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Full-Bodied")
                     

KeyboardInterrupt: 

In [101]:
''' Types of Wines in Data 
[' Red ', ' White ', ' White Blend', ' Red Blend',
       'Sparkling Red Blend', ' Rosé ', 'Sparkling White Blend',
       'Sparkling White ', ' Rosé Blend', 'Sparkling Red ']
'''
#for i in range(len(data)):
#    if "Blend" in data["type"][i]:
#        print(data["category"][i])
for i in range(len(data)):
    if "Rosé" in data["type"][i]:
        print(i, data["description"][i])

61 ['pale', 'copper', 'hue', 'wine', 'passion', 'fruit', 'palate', 'muskmelon', 'crisp', 'racy', 'medium', 'body', 'summertime', 'refreshment', 'drink']
214 ['easy', 'selection', 'streak', 'soft', 'red', 'currant', 'cherry', 'core', 'wine', 'light', 'breezy', 'palate', 'thin', 'fleeting', 'finish']
215 ['wine', 'earthy', 'rustic', 'fruit', 'spicy', 'pepper', 'edge', 'chill']
275 ['blossom', 'berry', 'nut', 'light-bodied', 'wine', 'zesty', 'lithe', 'palate', 'ideal', 'summer-to-fall', 'apéritif', 'porch', 'sipper', 'finish', 'hint', 'waxy']
331 ['big', 'spicy', 'wine', 'ripe', 'red', 'ss', 'pepper', 'tight', 'lemon-zest', 'acidity']
332 ['watermelon', 'natural', 'vanilla', 'mark', 'bouquet', 'palate', 'fleshy', 'crisp', 'nectarine', 'red', 'apple', 'strawberry', 'finish', 'sweetness']
382 ['remote', 'estate', 'edge', 'lubéron', 'north', 'provence', 'wine', 'rich', 'great', 'concentration', 'structure', 'cabernet', 'sauvignon', 'syrah', 'blend', 'full', 'ripe', 'red', 'dense', 'end']
384

2761 ['attractive', 'family-owned', 'estate', 'fruity', 'crisp', 'light', 'wine', 'orange-zest', 'acidity', 'citrus', 'tight', 'mineral', 'character', 'light', 'wine—bone', 'dry', 'touch', 'pepper', 'end']
2782 ['fresh', 'fruity', 'possible', 'wine', 'bottle', 'stylish', 'table', 'fragrant', 'red', 'apéritif']
2790 ['cherry', 'raspberry', 'fresh', 'clean', 'simple', 'juicy', 'direct', 'palate', 'cherry', 'plum', 'skin', 'citric', 'acid', 'rosé', 'definition', 'simple', 'summer', 'quaffer']
2902 ['heavy', 'stalky', 'burnt', 'leaden', 'palate', 'weighty', 'short', 'freshness', 'pithy', 'stalky', 'nectarine', 'plum', 'thrown', 'fading']
2907 ['soft', 'sweet', 'raspberry', 'fruit-flavored', 'wine', 'line', 'acidity', 'main', 'character', 'light', 'sweet', 'aftertaste', 'screwcap']
2909 ['dry', 'side', 'caramel', 'strawberry', 'attractive', 'bright', 'rosé', 'fresh']
2932 ['fruity', 'wine', 'ripe', 'red', 'red', 'acidity', 'wine', 'weight', 'fresh', 'crisp', 'rosé', 'food', 'rich']
2933 ['s

5860 ['pink', 'grapefruit', 'lemon', 'rind', 'wet', 'stone', 'watermelon', 'background', 'palate', '%', 'merlot', 'rosé', 'citrus-driven', 'impression', 'roundness', 'tart', 'peach', 'melon']
5868 ['strawberry', 'raspberry', 'peppery', 'medium-bodied', 'palate', 'driven', 'mulberry', 'stone-fruit', 'citrus', 'verve']
5871 ['group', 'chateaus', 'sumeire', 'family', 'coussin', 'second', 'wine', 'tangy', 'acidity', 'texture', 'important', 'fruit', 'mineral', 'edge', 'acidity', 'red', 'finish', 'attractive', 'lift']
5877 ['bit', 'shy', 'bouquet', 'concentration', 'cherry-kissed', 'rosé', 'summer', 'sip', 'blend', 'pinot', 'noir', 'dornfelder', 'touch', 'tart', 'brisk', 'zippy', 'finish']
5887 ['fresh', 'wine', 'cleanest', 'acidity', 'attractive', 'red', 'grapefruit', 'juice', 'great', 'apéritif', 'style', 'screwcap']
5889 ['clean', 'fresh', 'crisp', 'body', 'green', 'herbal', 'red', 'plum', 'juice', 'finish', 'acidity']
5920 ['showy', 'exuberant', 'wine', 'hot', 'pink', 'color', 'fresh', '

9782 ['pale', 'copper', 'wine', 'mineral', 'fruit', 'expression', 'wet', 'stone', 'front', 'citrus', 'rind', 'medium-bodied', 'dance', 'palate', 'show', 'concentration', 'texture', 'balance']
9787 ['domaine', 'la', 'mordorée', 'rosés', 'fitting', 'proprietor', 'christophe', 'delorme', 'final', 'vintage', 'june', 'blend', '%', 'grenache', '%', 'syrah', '%', 'cinsault', 'weighty', 'rich', 'blending', 'white', 'chocolate', 'wine', 'light', 'red', 'wine', 'poultry']
9801 ['crisp', 'dry', 'rosé', 'mourvèdre', 'grenache', 'great', 'acidity', 'lean', 'thirst-quenching', 'texture', 'vibrant', 'raspberry', 'apple', 'right', 'color', 'pale', 'salmon', 'body', 'light', 'finish', 'fresh', 'clean', 'perfect', 'lunch', 'wine', 'seafood', 'fare']
9819 ['négociant', 'fine', 'rosé', 'deep', 'pink', 'color', 'pepper', 'ample', 'body', 'cardamom', 'cinnamon', 'watermelon', 'red', 'palate', 'drink']
9835 ['yellow', 'orange', 'tan', 'pink', 'glass', 'blend', '%', 'grenache', '%', 'mourvèdre', '%', 'counois

11864 ['small', 'loire', 'valley', 'crisp', 'wine', 'pineau', "d'aunis", 'bright', 'fresh', 'acidity', 'red', 'currant', 'light', 'tight', 'aftertaste', 'drink']
11969 ['pink', 'color', 'smooth', 'wine', 'caramel', 'real', 'fruit']
11970 ['darker', 'color', 'normal', 'rosé', 'bordeaux', 'clairet', 'richness', 'wine', 'vanilla', 'sweet', 'strawberry', 'lack', 'freshness', 'treat', 'red', 'wine']
12019 ['estate', 'vineyard', 'red', 'mountain', 'exceptional', 'rosé', 'counoise', 'cinsault', 'grenache', 'pale', 'coppery', 'pink', 'shade', 'light', 'wine', 'fragrant', 'full-bodied', 'salmonberry', 'strawberry', 'fruit', 'light', 'spice', 'excellent', 'length']
12033 ['tall', 'square', 'bottle', 'presentation', 'wine', 'sense', 'presence', 'ripe', 'full-bodied', 'structure', 'full', 'red', 'light', 'acidity', 'fruitiness', 'delicious', 'apéritif']
12034 ['quat', 'meaning', 'wine', 'summer', 'food', 'wine', 'dry', 'full-bodied', 'character', 'ripe', 'caramel', 'red', 'plum', 'fresh', 'end', '

KeyError: 13998

In [ ]:
for i in range(len(data)):
    if "Rosé" in data["type"][i]:
        print(i, data["category"][i])

In [157]:
#for i in range(len(data)):
#    data['category'][i] = set(data['category'][i])  

In [156]:
# Save back to CSV
data.to_csv("wine_final.csv", index=False)